In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/processed/global_cancer_patients_2015_2024.csv")

In [3]:
df.describe()

,Age,Genetic_Risk,Air_Pollution,Alcohol_Use,Smoking,Obesity_Level,Cancer_Stage,Target_Severity_Score
count,33505.000000,33505.000000,33505.000000,33505.000000,33505.000000,33505.000000,33505.000000,33505.000000
mean,54.365916,4.999516,5.019681,4.998517,4.988885,4.990804,2.002477,4.952900
std,20.153346,2.876429,2.889599,2.889678,2.881810,2.889941,1.410883,1.195847
min,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.900000
25%,37.000000,2.500000,2.500000,2.500000,2.500000,2.500000,1.000000,4.120000
50%,54.000000,5.000000,5.000000,5.000000,5.000000,5.000000,2.000000,4.950000
75%,72.000000,7.500000,7.500000,7.500000,7.500000,7.500000,3.000000,5.790000
max,89.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000,8.860000


In [4]:
df.head()

,Age,Gender,Genetic_Risk,Air_Pollution,Alcohol_Use,Smoking,Obesity_Level,Cancer_Type,Cancer_Stage,Target_Severity_Score
0,71,Male,6.4,2.8,9.5,0.9,8.7,Lung,3,4.92
1,34,Male,1.3,4.5,3.7,3.9,6.3,Leukemia,0,4.65
2,80,Male,7.4,7.9,2.4,4.7,0.1,Breast,2,5.84
3,40,Male,1.7,2.9,4.8,3.5,2.7,Colon,1,3.12
4,43,Female,5.1,2.8,2.3,6.7,0.5,Skin,3,3.62
